In [1]:
!pip install lmdb

In [3]:
import lmdb
import json

from utils import *

track_json_path = 'json/msd_reviews.json'

with open(track_json_path) as json_file:
    track_json = json.load(json_file)

In [1]:
from vocab import Vocabulary

# sentences = []

# for item in track_json:
#     sentences.append(item['review_summary'])

# vocab = Vocabulary(sentences, vocab_size = 25000)
#vocab.save_vocab("data/vocab.json")

vocab = Vocabulary([], load_path="data/vocab.json")

In [5]:
SEQ_MAX_LEN = 64
root_dir='../msd/data/'

def convert_sentence(sentence):
    tokens = tokenize(sentence)[:SEQ_MAX_LEN]
    return  [vocab.stoi[tok] if tok in vocab.itos
              else vocab.stoi['<unk>']
              for tok in tokens]

def create_item(item):
    
    sample = {}
    
    sample['summary'] = convert_sentence(item['review_summary'])
    sample['full_text'] =  convert_sentence(item['review_text'])
    
    path = get_path_from_id(item['track_id'], root_dir)
    song_features = get_features(path).astype(np.float32)
    
    sample['song_features'] = song_features.tolist()
    
    return json.dumps(sample).encode()

tester = create_item(track_json[0])

In [6]:
import sys

obj = json.loads(tester)

np.array(obj["song_features"]).shape

(492, 24)

In [7]:
import tqdm.notebook as tqdm

env = lmdb.open('data/MSD_Echonest_Reviews', map_size=len(track_json)*sys.getsizeof(create_item(track_json[0])))

block = []
i = 0
for idx, it in enumerate(tqdm.tqdm(track_json[:10000])):
    block.append(it)
    
    if idx % 1000 == 999:
        with env.begin(write=True) as txn:
            for item in block:
                str_id = '{:08}'.format(i).encode("ascii")
                i += 1
                txn.put(str_id, create_item(item))
                
        block = []
        

In [13]:
env = lmdb.open('data/MSD_Echonest_Reviews', readonly=True)
with env.begin() as txn:
    length = txn.stat()['entries']
    
length

2242000

10000

In [2]:
env = lmdb.open('data/MSD_Echonest_Reviews')

env.stat()

{'psize': 4096,
 'depth': 3,
 'branch_pages': 66,
 'leaf_pages': 14509,
 'overflow_pages': 127083995,
 'entries': 2242000}